In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import functools as ft
import multiprocessing as mp
import os.path as op
import sys

In [3]:
%matplotlib inline

In [4]:
from hgflask import app
import hgflask
from flask import jsonify, request

In [5]:
import hgtiles

processes = {}

### HiGlass Server

In [44]:
base_cnv_path = '/Users/peter/Dropbox/paper-data/2017-10-24-Visidro/CNV_files/'

hgflask.TILESETS = [{  
    'uuid': "abc",
    'filetype': "bigwig",
    'datatype': "vector",
    'url': 'http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeSydhTfbs/wgEncodeSydhTfbsGm12878Ctcfsc15914c20StdSig.bigWig',
    'private': False,
    'name': "blabla",
    'coordSystem': "hg19",
    'coordSystem2': "hg19",
    'created': "2017-02-02T21:10:56.265085Z",
    },
    {
        'uuid': 'id_arcs',
        "name": "my cooler",
        'filetype': 'bedarcsdb',
        'datatype': 'arcs',
        #'filepath': op.expanduser('~/projects/hgtiles/data/arrowhead_domains_short.txt.multires.db'),
        'filepath': op.expanduser('/Users/peter/Dropbox/paper-data/2017-10-24-Visidro/SV_calls_1.6/9ae0744a-9bc1-4cd7-b7cf-c6569ed9e4aa.pcawg_consensus_1.6.161022.somatic.sv.bedpe.multires.db'),
        #'filepath': op.expanduser('/Users/peter/Dropbox/paper-data/2017-10-24-Visidro/SV_calls_1.6/72f0a49a-aec8-47e5-846a-956c4da1507c.pcawg_consensus_1.6.161022.somatic.sv.bedpe.multires.db'),
        #'filepath': op.expanduser('/Users/peter/Dropbox/paper-data/2017-10-24-Visidro/SV_calls_1.6/ea43434b-197e-48ac-ae2e-46bc7f3776de.pcawg_consensus_1.6.161022.somatic.sv.bedpe.multires.db'),

    },
    {
        'uuid': 'cnvs',
        "name": "cnvs",
        'filetype': 'beddb',
        'datatype': 'bedlike',
        #'filepath': op.expanduser('~/projects/hgtiles/data/arrowhead_domains_short.txt.multires.db'),
        'filepath': op.expanduser(op.join(
        base_cnv_path,
            '3e86e68b-d083-4450-b056-4cd0f8d4a084.consensus.20170119.somatic.cna.annotated.txt.beddb')),
        #'filepath': op.expanduser('/Users/peter/Dropbox/paper-data/2017-10-24-Visidro/SV_calls_1.6/72f0a49a-aec8-47e5-846a-956c4da1507c.pcawg_consensus_1.6.161022.somatic.sv.bedpe.multires.db'),
        #'filepath': op.expanduser('/Users/peter/Dropbox/paper-data/2017-10-24-Visidro/SV_calls_1.6/ea43434b-197e-48ac-ae2e-46bc7f3776de.pcawg_consensus_1.6.161022.somatic.sv.bedpe.multires.db'),

    }
]

import slugid

for puid in processes:
    processes[puid].terminate()

processes = {}
print(hgflask.TILESETS)
# we're going to assign a uuid to each server process so that if anything
# goes wrong, the variable referencing the process doesn't get lost
uuid = slugid.nice()
processes[uuid] = mp.Process(target=ft.partial(app.run, debug=True, port=4008, host='0.0.0.0', use_reloader=False))
processes[uuid].start()

[{'uuid': 'abc', 'filetype': 'bigwig', 'datatype': 'vector', 'url': 'http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeSydhTfbs/wgEncodeSydhTfbsGm12878Ctcfsc15914c20StdSig.bigWig', 'private': False, 'name': 'blabla', 'coordSystem': 'hg19', 'coordSystem2': 'hg19', 'created': '2017-02-02T21:10:56.265085Z'}, {'uuid': 'id_arcs', 'name': 'my cooler', 'filetype': 'bedarcsdb', 'datatype': 'arcs', 'filepath': '/Users/peter/Dropbox/paper-data/2017-10-24-Visidro/SV_calls_1.6/9ae0744a-9bc1-4cd7-b7cf-c6569ed9e4aa.pcawg_consensus_1.6.161022.somatic.sv.bedpe.multires.db'}, {'uuid': 'cnvs', 'name': 'cnvs', 'filetype': 'beddb', 'datatype': 'bedlike', 'filepath': '/Users/peter/Dropbox/paper-data/2017-10-24-Visidro/CNV_files/3e86e68b-d083-4450-b056-4cd0f8d4a084.consensus.20170119.somatic.cna.annotated.txt.beddb'}]


 * Running on http://0.0.0.0:4008/ (Press CTRL+C to quit)


new_rows length 238


127.0.0.1 - - [22/Jun/2018 11:26:36] "GET /api/v1/tiles/?d=cnvs.2.2 HTTP/1.1" 200 -


ts: {'uuid': 'cnvs', 'name': 'cnvs', 'filetype': 'beddb', 'datatype': 'bedlike', 'filepath': '/Users/peter/Dropbox/paper-data/2017-10-24-Visidro/CNV_files/3e86e68b-d083-4450-b056-4cd0f8d4a084.consensus.20170119.somatic.cna.annotated.txt.beddb'}
filetype beddb


...

127.0.0.1 - - [22/Jun/2018 11:29:24] "GET /api/v1/tiles/?d=cnvs.1.0&d=cnvs.1.1&s=bw63WbGvTZaha5t1sgqrlw HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2018 11:37:10] "OPTIONS /api/v1/tileset_info/?d=cnvs&s=c78rb5m2RZahply6s8kBtQ HTTP/1.1" 200 -


ts: {'uuid': 'cnvs', 'name': 'cnvs', 'filetype': 'beddb', 'datatype': 'bedlike', 'filepath': '/Users/peter/Dropbox/paper-data/2017-10-24-Visidro/CNV_files/3e86e68b-d083-4450-b056-4cd0f8d4a084.consensus.20170119.somatic.cna.annotated.txt.beddb'}
filetype beddb


127.0.0.1 - - [22/Jun/2018 11:37:10] "GET /api/v1/tileset_info/?d=cnvs&s=c78rb5m2RZahply6s8kBtQ HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2018 11:37:11] "OPTIONS /api/v1/tiles/?d=cnvs.1.0&d=cnvs.1.1&s=c78rb5m2RZahply6s8kBtQ HTTP/1.1" 200 -


new_rows length 884
new_rows length 238


127.0.0.1 - - [22/Jun/2018 11:37:11] "GET /api/v1/tiles/?d=cnvs.1.0&d=cnvs.1.1&s=c78rb5m2RZahply6s8kBtQ HTTP/1.1" 200 -


In [45]:
!curl localhost:4008/api/v1/tiles/?d=cnvs.2.2

{
  "cnvs.2.2": [
    {
      "chrOffset": 2581367074, 
      "fields": [
        "18", 
        "53209919", 
        "57749998", 
        "2", 
        "1", 
        "1", 
        "3", 
        "b", 
        "5", 
        "1", 
        "1", 
        "0.0422307714787", 
        "0.0604959913739", 
        "2", 
        "1", 
        "1", 
        "1", 
        "1", 
        "1", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "1", 
        "1", 
        "1", 
        "NA", 
        "NA", 
        "NA", 
        "2", 
        "1", 
        "1"
      ], 
      "importance": 4540079.0, 
      "uid": "IoKiRGjeR0ewgiHYL6vGNA", 
      "xEnd": 2639117072, 
      "xStart": 2634576993
    }, 
    {
      "chrOffset": 2581367074, 
      "fields": [
        "18", 
        "57749999", 
        "59968856", 
        "2", 
        "1", 
        "1", 
        "2", 
        "d", 
        "4", 

        "0.060839932062", 
        "2", 
        "1", 
        "1", 
        "1", 
        "1", 
        "1", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "2", 
        "1", 
        "1", 
        "NA", 
        "NA", 
        "NA", 
        "2", 
        "1", 
        "1"
      ], 
      "importance": 4646804.0, 
      "uid": "Ig1MVQJiToqgkTNuJKkVew", 
      "xEnd": 2473427060, 
      "xStart": 2468780256
    }, 
    {
      "chrOffset": 2409817111, 
      "fields": [
        "16", 
        "64461919", 
        "66276494", 
        "2", 
        "1", 
        "1", 
        "2", 
        "d", 
        "2", 
        "1", 
        "1", 
        "0.0438399018809", 
        "0.0708603831436", 
        "2.369715", 
        "1.1848576", 
        "1.1848576", 
        "1", 
        "1", 
        "0.953139643861294", 
        "2", 
        "1", 
        "0.046860356138706", 
      

In [46]:
!curl localhost:4008/api/v1/tileset_info/?d=cnvs

{
  "cnvs": {
    "assembly": "b37", 
    "chrom_names": "1\t2\t3\t4\t5\t6\t7\t8\t9\t10\t11\t12\t13\t14\t15\t16\t17\t18\t19\t20\t21\t22\tX\tY\tMT", 
    "chrom_sizes": "249250621\t243199373\t198022430\t191154276\t180915260\t171115067\t159138663\t146364022\t141213431\t135534747\t135006516\t133851895\t115169878\t107349540\t102531392\t90354753\t81195210\t78077248\t59128983\t63025520\t48129895\t51304566\t155270560\t59373566\t16569", 
    "datatype": "bedlike", 
    "filepath": "/Users/peter/Dropbox/paper-data/2017-10-24-Visidro/CNV_files/3e86e68b-d083-4450-b056-4cd0f8d4a084.consensus.20170119.somatic.cna.annotated.txt.beddb", 
    "filetype": "beddb", 
    "header": "chromosome\tstart\tend\ttotal_cn\tmajor_cn\tminor_cn\tstar\tlevel\tmethods_agree\tabsolute_broad_major_cn\tabsolute_broad_minor_cn\tabsolute_broad_het_error\tabsolute_broad_cov_error\taceseq_copy_number\taceseq_minor_cn\taceseq_major_cn\tbattenberg_nMaj1_A\tbattenberg_nMin1_A\tbattenberg_frac1_A\tbattenberg_nMaj2_A\tbattenberg

{
  "cnvs.0.0": [
    {
      "chrOffset": 0, 
      "fields": [
        "1", 
        "1", 
        "121499999", 
        "2", 
        "1", 
        "1", 
        "3", 
        "a", 
        "6", 
        "1", 
        "1", 
        "0.0385702148217", 
        "0.0687900564453", 
        "2", 
        "1", 
        "1", 
        "1", 
        "1", 
        "1", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "NA", 
        "2", 
        "1", 
        "1", 
        "1", 
        "1", 
        "1", 
        "NA", 
        "NA", 
        "NA", 
        "2", 
        "1", 
        "1"
      ], 
      "importance": 121499998.0, 
      "uid": "HJDHUFipSsmhcwh-rCaYHQ", 
      "xEnd": 121499999, 
      "xStart": 1
    }, 
    {
      "chrOffset": 0, 
      "fields": [
        "1", 
        "121500000", 
        "128899999", 
        "2", 
        "1", 
        "1", 
        "3", 
        "b", 
        "6", 
        "1", 
        "1", 
      

In [21]:
!curl higlass.io/api/v1/tiles/?d=CQMd6V_cRw6iCI_-Unl3PQ.0.0.0

{"CQMd6V_cRw6iCI_-Unl3PQ.0.0.0": {"min_value": 0.0, "max_value": 1.2987070083618164, "dense": "...

...", "dtype": "float16"}}

In [21]:
a = np.random.random((4,3))
print(a)

[[ 0.04855233  0.75723857  0.08055191]
 [ 0.1713757   0.58781498  0.01928301]
 [ 0.64798268  0.37728143  0.954238  ]
 [ 0.55084939  0.95120137  0.56308623]]


In [23]:
a.T.reshape((a.shape[1],-1,2)).reduce()

AttributeError: 'numpy.ndarray' object has no attribute 'reduce'

In [25]:
a = np.array([1,2,3,4])
a.reshape((-1,2)).sum(axis=1)

array([3, 7])

In [26]:
a.reshape((-1,2)).max(axis=1)

array([2, 4])

In [27]:
b = np.array([[1,2],[3,4],[5,6],[3,3]])

In [29]:
b = [[1,2],[3,4],[5,6],[3,3]]
c = []
for i in range(0,len(b) - 1,2):
    if sum(b[i]) > sum(b[i+1]):
        c += [b[i]]
    else:
        c += [b[i+1]]
print(c)

[[3, 4], [5, 6]]
